<a href="https://colab.research.google.com/github/sergio-cabrales/python/blob/main/MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import the libraries
import pandas as pd
import pandas_datareader as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [119]:
# Get the stock quote
df = web.DataReader('AAPL', data_source='yahoo',start='2020-01-01', end='2021-01-01')
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-01-02,75.150002,73.797501,74.059998,75.087502,135480400.0,74.333511
2020-01-03,75.144997,74.125000,74.287498,74.357498,146322800.0,73.610840
2020-01-06,74.989998,73.187500,73.447502,74.949997,118387200.0,74.197395
2020-01-07,75.224998,74.370003,74.959999,74.597504,108872000.0,73.848442
2020-01-08,76.110001,74.290001,74.290001,75.797501,132079200.0,75.036385
...,...,...,...,...,...,...
2020-12-24,133.460007,131.100006,131.320007,131.970001,54930100.0,131.773087
2020-12-28,137.339996,133.509995,133.990005,136.690002,124486200.0,136.486053
2020-12-29,138.789993,134.339996,138.050003,134.869995,121047300.0,134.668762


In [95]:
#load the  data set
# from google.colab import files
# files.upload()

In [96]:
# df pd.read_csv('GOOG.csv')
# df

In [120]:
# create functions to calculate the Simple Moving Average (SMA) & th exponential Moving Average (EMA)
# Create the SMA
def SMA(data, period=30, column ='Adj Close'):
  return data[column].rolling(window=period).mean()
# Create the EMA
def EMA(data, period=20, column ='Adj Close'):
  return data[column].ewm(span=period, adjust=False).mean()

In [121]:
#Create a function to calculate the Moving Average Convergence / Divergence (MACD)
def MACD(data, period_long=26, period_short=12, period_signal=9, column='Adj Close'):
  # Calculate the short Term EMA
  ShortEMA = EMA(data, period=period_short, column=column)
  # Calculate the long Term EMA
  LongEMA = EMA(data, period=period_long, column=column)
  # Calculate MACD
  data['MACD'] = ShortEMA - LongEMA
  # Calculate the signal line and store it into the data frama
  data['signal_line'] = EMA(data, period=period_signal, column='MACD')

  return data

In [122]:
# Creat a function to calculate the Relative Strenght Index (RSI)
def RSI(data,period=14, column ='Adj Close'):
  delta = data[column].diff(1)
  detal = delta.dropna()
  up = delta.copy()
  down = delta.copy()
  up[up < 0] = 0
  down[down >0]
  data['up'] = up
  data['down'] = down
  AVG_Gain = SMA(data, period, column='up')
  AVG_Loss = abs(SMA(data,period, column='down'))
  RS = AVG_Gain / AVG_Loss
  RSI = 100 - (100/(1+RS))

  data['RSI'] = RSI
  return data


In [123]:
# Add the indicators to data set
MACD(df)
RSI(df)
df['SMA'] = SMA(df)
df['EMA'] = EMA(df)
#Show data
df

,High,Low,Open,Close,Volume,Adj Close,MACD,signal_line,up,down,RSI,SMA,EMA
Date,,,,,,,,,,,,,
2020-01-02,75.150002,73.797501,74.059998,75.087502,135480400.0,74.333511,0.000000,0.000000,NaN,NaN,NaN,NaN,74.333511
2020-01-03,75.144997,74.125000,74.287498,74.357498,146322800.0,73.610840,-0.057649,-0.011530,0.000000,-0.722672,NaN,NaN,74.264685
2020-01-06,74.989998,73.187500,73.447502,74.949997,118387200.0,74.197395,-0.055368,-0.020297,0.586555,0.586555,NaN,NaN,74.258277
2020-01-07,75.224998,74.370003,74.959999,74.597504,108872000.0,73.848442,-0.080786,-0.032395,0.000000,-0.348953,NaN,NaN,74.219245
2020-01-08,76.110001,74.290001,74.290001,75.797501,132079200.0,75.036385,-0.005016,-0.026919,1.187943,1.187943,NaN,NaN,74.297068
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,133.460007,131.100006,131.320007,131.970001,54930100.0,131.773087,3.332375,2.680560,1.008484,1.008484,63.362229,122.326539,125.378036
2020-12-28,137.339996,133.509995,133.990005,136.690002,124486200.0,136.486053,3.801722,2.904792,4.712967,4.712967,60.752186,122.908336,126.435942
2020-12-29,138.789993,134.339996,138.050003,134.869995,121047300.0,134.668762,3.981151,3.120064,0.000000,-1.817291,64.904649,123.427893,127.220020


In [124]:
# Create the Target column
df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
# Show the data
df

,High,Low,Open,Close,Volume,Adj Close,MACD,signal_line,up,down,RSI,SMA,EMA,Target
Date,,,,,,,,,,,,,,
2020-01-02,75.150002,73.797501,74.059998,75.087502,135480400.0,74.333511,0.000000,0.000000,NaN,NaN,NaN,NaN,74.333511,0
2020-01-03,75.144997,74.125000,74.287498,74.357498,146322800.0,73.610840,-0.057649,-0.011530,0.000000,-0.722672,NaN,NaN,74.264685,1
2020-01-06,74.989998,73.187500,73.447502,74.949997,118387200.0,74.197395,-0.055368,-0.020297,0.586555,0.586555,NaN,NaN,74.258277,0
2020-01-07,75.224998,74.370003,74.959999,74.597504,108872000.0,73.848442,-0.080786,-0.032395,0.000000,-0.348953,NaN,NaN,74.219245,1
2020-01-08,76.110001,74.290001,74.290001,75.797501,132079200.0,75.036385,-0.005016,-0.026919,1.187943,1.187943,NaN,NaN,74.297068,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,133.460007,131.100006,131.320007,131.970001,54930100.0,131.773087,3.332375,2.680560,1.008484,1.008484,63.362229,122.326539,125.378036,1
2020-12-28,137.339996,133.509995,133.990005,136.690002,124486200.0,136.486053,3.801722,2.904792,4.712967,4.712967,60.752186,122.908336,126.435942,0
2020-12-29,138.789993,134.339996,138.050003,134.869995,121047300.0,134.668762,3.981151,3.120064,0.000000,-1.817291,64.904649,123.427893,127.220020,0


In [125]:
#Remove the first 29 days of data
df = df[29:]
df

,High,Low,Open,Close,Volume,Adj Close,MACD,signal_line,up,down,RSI,SMA,EMA,Target
Date,,,,,,,,,,,,,,
2020-02-13,81.555000,80.837502,81.047501,81.217499,94747600.0,80.592773,1.080537,0.994861,0.000000,-0.578026,84.854174,77.889106,78.835143,1
2020-02-14,81.495003,80.712502,81.184998,81.237503,80113600.0,80.612625,1.090336,1.013956,0.019852,0.019852,71.045716,78.098410,79.004427,0
2020-02-18,79.937500,78.652496,78.839996,79.750000,152531200.0,79.136566,0.967840,1.004733,0.000000,-1.476059,94.004871,78.282601,79.017012,1
2020-02-19,81.142502,80.000000,80.000000,80.904999,93984000.0,80.282684,0.952267,0.994240,1.146118,1.146118,99.842122,78.485444,79.137552,0
2020-02-20,81.162498,79.552498,80.657501,80.074997,100566000.0,79.459068,0.863511,0.968094,0.000000,-0.823616,91.537325,78.672465,79.168172,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,133.460007,131.100006,131.320007,131.970001,54930100.0,131.773087,3.332375,2.680560,1.008484,1.008484,63.362229,122.326539,125.378036,1
2020-12-28,137.339996,133.509995,133.990005,136.690002,124486200.0,136.486053,3.801722,2.904792,4.712967,4.712967,60.752186,122.908336,126.435942,0
2020-12-29,138.789993,134.339996,138.050003,134.869995,121047300.0,134.668762,3.981151,3.120064,0.000000,-1.817291,64.904649,123.427893,127.220020,0


In [126]:
#Split tha data set into a feature or independent data set (X) and a Target or dependent data set (Y)
keep_columns = ['Adj Close', 'MACD', 'signal_line' , 'RSI', 'EMA']
X = df[keep_columns].values
Y = df['Target'].values


In [127]:
# Split the data again but this time into 80% trading and 20% test
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state =2)

In [128]:
# Create and train the decision tree classsifier model
tree = DecisionTreeClassifier().fit(X_train,Y_train)

In [129]:
#Check how well the model did on the trainign data set
print(tree.score(X_train,Y_train))

1.0


In [130]:
#Check how weel the model did on the testing data set
print(tree.score(X_test,Y_test))

0.4444444444444444


In [131]:
#Show the model tree predictions
tree_predictions = tree.predict(X_test)
print(tree_predictions)

[0 1 1 1 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0
 0 1 0 0 0 1 0 1]


In [132]:
Y_test

array([1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       1])

In [134]:
# Get the model metrics
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test, tree_predictions))
print(confusion_matrix(Y_test,tree_predictions))

              precision    recall  f1-score   support

           0       0.35      0.44      0.39        18
           1       0.55      0.44      0.49        27

    accuracy                           0.44        45
   macro avg       0.45      0.44      0.44        45
weighted avg       0.47      0.44      0.45        45

[[ 8 10]
 [15 12]]


In [133]:
#A support vector machine (SVM) is a type of supervised machine learning classification algorithm.
from sklearn.svm import SVC
# svclassifier = SVC(kernel='linear')
# Polynomial Kernel
svclassifier  = SVC(kernel='poly', degree=8)
# Gaussian Kernel
#svclassifier = SVC(kernel='rbf')
# Sigmoid Kernel
svclassifier = SVC(kernel='sigmoid')
svclassifier.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [135]:
tree_predictions = svclassifier.predict(X_test)

In [136]:
# Get the model metrics
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test, tree_predictions))
print(confusion_matrix(Y_test,tree_predictions))

              precision    recall  f1-score   support

           0       0.50      0.06      0.10        18
           1       0.60      0.96      0.74        27

    accuracy                           0.60        45
   macro avg       0.55      0.51      0.42        45
weighted avg       0.56      0.60      0.49        45

[[ 1 17]
 [ 1 26]]


In [137]:
#Check how weel the model did on the testing data set
print(svclassifier.score(X_test,Y_test))

0.6
